In [ ]:
from preprocess import dataset,get_Kfold,get_scheduler
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from models import FM_MLP,AE_MLP,get_models
from torch import optim
from torch.utils.data import DataLoader,Dataset
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(1)

# Config

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/useful totaldata.xlsx'
data = pd.read_excel(file,engine="openpyxl")
data = np.array(data)
para_path = './save/parameter'
batch_size = 300
flag = 0
plot = True
np.set_printoptions(threshold=np.sys.maxsize)

# Train

In [ ]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
total_mse_list = []
for config in config_list:
    Type = config['Type']
    N_size = config['N_size']
    num_epoch = config['num_epoch']
    lr = config['lr']
    decay = config["decay"]
    k = config['k']
    alpha = config['alpha']
    input_size = config['input_size']
    L1,L2 = config['L1'],config['L2']
    
    K_fold_data = get_Kfold(file,k)
    for k_step,(trian_data,test_data) in enumerate(K_fold_data):
        tqdm.write(f"-----------[ Times:{flag+1} K:{k_step+1} Type:{Type} N_size:{N_size} ]-----------")
        model = get_models(config['model'],input_size,L1,L2).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr=lr,weight_decay=decay)
        lr_scheduler = get_scheduler(optimizer,num_epoch,flag)
        scaler = MinMaxScaler()
        
        #get data prepared
        trian_data = dataset(trian_data,Type,N_size,input_size,scaler)
        test_data = dataset(test_data,Type,N_size,input_size,scaler)
        
        train_loader = DataLoader(dataset=trian_data,batch_size=batch_size)
        test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
        
        #train
        train_MLP(train_loader,
                  model,
                  criterion,
                  alpha,
                  optimizer,
                  num_epoch,
                  k_step,
                  lr_scheduler,
                  para_path,
                  device,
                  flag)
        
        #tese
        sum_mse,rmse = test_MLP(test_loader,
                             model,
                             k_step,
                             criterion,
                             alpha,
 #                            plot,
                             0,
                             para_path,
                             device,
                             flag)
        total_mse_list.append(sum_mse)
    flag +=1
    tqdm.write(f"total rmse of k fold is: {rmse}")

total_rmse = (sum(total_mse_list)/len(data))**0.5
print(f"total rmse of all data is: {total_rmse}")